# setup

In [1]:
import re
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import types, Row, Column
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer

In [2]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "finalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

# check file size

In [3]:
!wc -l data/train.txt

45840617 data/train.txt


In [4]:
!wc -l data/test.txt

6042135 data/test.txt


# experimenting and some EDA with DFs

In [5]:
train = spark.read.option('header', 'false').csv('data/train.txt', sep='\t')

In [6]:
train.write.format('parquet').save('data/train.parquet')
train_parquet = spark.read.parquet('data/train.parquet')

In [9]:
!du data/train.txt
!du data/train.parquet

10885924	data/train.txt
2895045	data/train.parquet


In [7]:
%%time
train.count()

CPU times: user 20 ms, sys: 60 ms, total: 80 ms
Wall time: 2min 47s


45840617

In [8]:
%%time
train_parquet.count()

CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 2.85 s


45840617

In [15]:
test = spark.read.option('header','false').csv('data/test.txt', sep='\t')
test.write.format('parquet').save('data/test.parquet')
del test
# test_parquet = spark.read.parquet('data/test.parquet')

In [12]:
# %%time
# test_parquet.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 327 ms


6042135

In [3]:
train = spark.read.parquet('data/train.parquet')
# s = train.sample(False, 0.001)

In [11]:
# test = spark.read.parquet('data/test.parquet')

In [4]:
# df = spark.read.option('header', 'false').csv('data/toy_train.txt', sep='\t')
# df.head()

In [5]:
# %%time
# df.count()

In [6]:
# df.select(df.columns[2]).show(5)

In [7]:
# # df.write.format('parquet').save('data/toy_train.parquet')
# df = spark.read.parquet('data/toy_train.parquet')
# df.printSchema()

In [9]:
# for c in df.columns:
#     df = df.withColumnRenamed(c, c.strip('_'))
# df.columns[:5]

In [8]:
# for c in df.columns[:14]:
#     df = df.withColumn(c, df[c].cast('float'))
# df.printSchema()

In [6]:
# full training set
for c in train.columns:
    train = train.withColumnRenamed(c, c.strip('_'))
train.columns[:5]

['c0', 'c1', 'c2', 'c3', 'c4']

In [7]:
# full training set
for c in train.columns[:14]:
    train = train.withColumn(c, train[c].cast('float'))
train.printSchema()

root
 |-- c0: float (nullable = true)
 |-- c1: float (nullable = true)
 |-- c2: float (nullable = true)
 |-- c3: float (nullable = true)
 |-- c4: float (nullable = true)
 |-- c5: float (nullable = true)
 |-- c6: float (nullable = true)
 |-- c7: float (nullable = true)
 |-- c8: float (nullable = true)
 |-- c9: float (nullable = true)
 |-- c10: float (nullable = true)
 |-- c11: float (nullable = true)
 |-- c12: float (nullable = true)
 |-- c13: float (nullable = true)
 |-- c14: string (nullable = true)
 |-- c15: string (nullable = true)
 |-- c16: string (nullable = true)
 |-- c17: string (nullable = true)
 |-- c18: string (nullable = true)
 |-- c19: string (nullable = true)
 |-- c20: string (nullable = true)
 |-- c21: string (nullable = true)
 |-- c22: string (nullable = true)
 |-- c23: string (nullable = true)
 |-- c24: string (nullable = true)
 |-- c25: string (nullable = true)
 |-- c26: string (nullable = true)
 |-- c27: string (nullable = true)
 |-- c28: string (nullable = true)
 |--

# Train/Test split

In [98]:
%%time
# full training set
stats = train[train.columns[1:14]].describe()
means = np.array(stats[stats['summary'] == 'mean'].collect())[0][1:]
stdevs = np.array(stats[stats['summary'] == 'stddev'].collect())[0][1:]

CPU times: user 70 ms, sys: 30 ms, total: 100 ms
Wall time: 1min 59s


In [1]:
%%time
# sample
stats = s[s.columns[1:14]].describe()
means = np.array(stats[stats['summary'] == 'mean'].collect())[0][1:]
stdevs = np.array(stats[stats['summary'] == 'stddev'].collect())[0][1:]

NameError: name 's' is not defined

In [99]:
stats.show(vertical=True)

-RECORD 0---------------------
 summary | count              
 c1      | 25047061           
 c2      | 45840617           
 c3      | 36001170           
 c4      | 35903248           
 c5      | 44657500           
 c6      | 35588289           
 c7      | 43857751           
 c8      | 45817844           
 c9      | 43857751           
 c10     | 25047061           
 c11     | 43857751           
 c12     | 10768965           
 c13     | 35903248           
-RECORD 1---------------------
 summary | mean               
 c1      | 3.5024133170754044 
 c2      | 105.84841979766546 
 c3      | 26.913041020611274 
 c4      | 7.322680248873305  
 c5      | 18538.991664871523 
 c6      | 116.06185085211598 
 c7      | 16.333130032135028 
 c8      | 12.517042137556713 
 c9      | 106.1098234380509  
 c10     | 0.6175294977722137 
 c11     | 2.7328343170173044 
 c12     | 0.9910356287721244 
 c13     | 8.217461161174054  
-RECORD 2---------------------
 summary | stddev             
 c1     

In [6]:
stats2 = train[train.columns[1:]].summary('count')

In [7]:
stats2.show(vertical=True)

-RECORD 0-----------
 summary | count    
 c1      | 25047061 
 c2      | 45840617 
 c3      | 36001170 
 c4      | 35903248 
 c5      | 44657500 
 c6      | 35588289 
 c7      | 43857751 
 c8      | 45817844 
 c9      | 43857751 
 c10     | 25047061 
 c11     | 43857751 
 c12     | 10768965 
 c13     | 35903248 
 c14     | 45840617 
 c15     | 45840617 
 c16     | 44281144 
 c17     | 44281144 
 c18     | 45840617 
 c19     | 40299992 
 c20     | 45840617 
 c21     | 45840617 
 c22     | 45840617 
 c23     | 45840617 
 c24     | 45840617 
 c25     | 44281144 
 c26     | 45840617 
 c27     | 45840617 
 c28     | 45840617 
 c29     | 44281144 
 c30     | 45840617 
 c31     | 45840617 
 c32     | 25667759 
 c33     | 25667759 
 c34     | 44281144 
 c35     | 10885544 
 c36     | 45840617 
 c37     | 44281144 
 c38     | 25667759 
 c39     | 25667759 



In [9]:
N = train.count()
N

45840617

In [10]:
nans = {}
for c in stats2.columns[1:]:
    nans[c] = N - int(stats2.select(c).collect()[0][0])
nans

{'c1': 20793556,
 'c2': 0,
 'c3': 9839447,
 'c4': 9937369,
 'c5': 1183117,
 'c6': 10252328,
 'c7': 1982866,
 'c8': 22773,
 'c9': 1982866,
 'c10': 20793556,
 'c11': 1982866,
 'c12': 35071652,
 'c13': 9937369,
 'c14': 0,
 'c15': 0,
 'c16': 1559473,
 'c17': 1559473,
 'c18': 0,
 'c19': 5540625,
 'c20': 0,
 'c21': 0,
 'c22': 0,
 'c23': 0,
 'c24': 0,
 'c25': 1559473,
 'c26': 0,
 'c27': 0,
 'c28': 0,
 'c29': 1559473,
 'c30': 0,
 'c31': 0,
 'c32': 20172858,
 'c33': 20172858,
 'c34': 1559473,
 'c35': 34955073,
 'c36': 0,
 'c37': 1559473,
 'c38': 20172858,
 'c39': 20172858}

# Pre-processing

In [11]:
%%time
# get unique counts of categorical variables
uniqueCounts = {}
for c in train.columns[14:]:
    count = train.agg(F.countDistinct(c).alias('c')).collect()[0]['c']
    uniqueCounts[c] = count

{'c14': 1460, 'c15': 583, 'c16': 10131226, 'c17': 2202607, 'c18': 305, 'c19': 23, 'c20': 12517, 'c21': 633, 'c22': 3, 'c23': 93145, 'c24': 5683, 'c25': 8351592, 'c26': 3194, 'c27': 27, 'c28': 14992, 'c29': 5461305, 'c30': 10, 'c31': 5652, 'c32': 2172, 'c33': 3, 'c34': 7046546, 'c35': 17, 'c36': 15, 'c37': 286180, 'c38': 104, 'c39': 142571}
CPU times: user 270 ms, sys: 180 ms, total: 450 ms
Wall time: 3min 52s


In [13]:
uniqueCounts

{'c14': 1460,
 'c15': 583,
 'c16': 10131226,
 'c17': 2202607,
 'c18': 305,
 'c19': 23,
 'c20': 12517,
 'c21': 633,
 'c22': 3,
 'c23': 93145,
 'c24': 5683,
 'c25': 8351592,
 'c26': 3194,
 'c27': 27,
 'c28': 14992,
 'c29': 5461305,
 'c30': 10,
 'c31': 5652,
 'c32': 2172,
 'c33': 3,
 'c34': 7046546,
 'c35': 17,
 'c36': 15,
 'c37': 286180,
 'c38': 104,
 'c39': 142571}

In [8]:
# grab a small-ish sample
# s = train.select('c0','c2','c3','c22','c33').sample(False, 0.001)
s = train.sample(False, 0.001)
s.count()

45877

In [9]:
s.head(5)

[Row(c0=1.0, c1=3.0, c2=108.0, c3=12.0, c4=8.0, c5=18.0, c6=12.0, c7=3.0, c8=9.0, c9=8.0, c10=1.0, c11=1.0, c12=2.0, c13=8.0, c14='68fd1e64', c15='38a947a1', c16='0d595c43', c17='3952a0a2', c18='43b19349', c19='fe6b92e5', c20='b67de336', c21='0b153874', c22='a73ee510', c23='3b08e48b', c24='873e6871', c25='be326377', c26='4cc724bd', c27='b28479f6', c28='a897defd', c29='fadb1467', c30='27c07bd6', c31='fe182c3d', c32=None, c33=None, c34='c009e522', c35=None, c36='32c7478e', c37='cc159b1c', c38=None, c39=None),
 Row(c0=0.0, c1=None, c2=0.0, c3=1.0, c4=1.0, c5=10153.0, c6=66.0, c7=2.0, c8=2.0, c9=47.0, c10=None, c11=2.0, c12=None, c13=1.0, c14='8cf07265', c15='78ccd99e', c16='f715d8cc', c17='7c15fa92', c18='4cf72387', c19='7e0ccccf', c20='6af7dea8', c21='5b392875', c22='a73ee510', c23='7953ffe6', c24='1acca553', c25='b9bee1c2', c26='3691e3e3', c27='07d13a8f', c28='162f3329', c29='498519e1', c30='07c540c4', c31='e7e991cb', c32='04de9d96', c33='5840adea', c34='71b9f31a', c35=None, c36='423fab

In [10]:
%%time
# grab some stats for standardizing the numerical columns
stats4 = s[s.columns[1:14]].describe()
means = np.array(stats4[stats4['summary'] == 'mean'].collect())[0][1:]
stdevs = np.array(stats4[stats4['summary'] == 'stddev'].collect())[0][1:]

CPU times: user 30 ms, sys: 50 ms, total: 80 ms
Wall time: 27.8 s


In [11]:
# standardize the numerical columns
for i,c in enumerate(s.columns[1:14]):
    s = s.withColumn(c, (s[c] - means[i]) / stdevs[i])

In [19]:
s.select(s.columns[1:14]).show(vertical=True)

-RECORD 0-------------------
 c1  | -0.06104319928899... 
 c2  | 0.009567439505994422 
 c3  | -0.04067442450909083 
 c4  | 0.08572355279011215  
 c5  | -0.26554282181769306 
 c6  | -0.22548584412649042 
 c7  | -0.21048066958486944 
 c8  | -0.22799719207658325 
 c9  | -0.45478881667449267 
 c10 | 0.5628429015639078   
 c11 | -0.3276097260071061  
 c12 | 0.15213296349014394  
 c13 | -0.01001306443818... 
-RECORD 1-------------------
 c1  | null                 
 c2  | -0.27572008471363835 
 c3  | -0.07370318523326326 
 c4  | -0.7267175939998046  
 c5  | -0.12109496051919905 
 c6  | -0.11011283442338143 
 c7  | -0.22594950340596895 
 c8  | -0.687796615232404   
 c9  | -0.27145215522454785 
 c10 | null                 
 c11 | -0.14069476574725504 
 c12 | null                 
 c13 | -0.5318162882239682  
-RECORD 2-------------------
 c1  | -0.40454315773474303 
 c2  | -0.27572008471363835 
 c3  | -0.04968226834295604 
 c4  | 0.4339126157000765   
 c5  | -0.26531478375348233 
 c6  | 0.32787

In [13]:
s.columns

['c0',
 'c1',
 'c2',
 'c3',
 'c4',
 'c5',
 'c6',
 'c7',
 'c8',
 'c9',
 'c10',
 'c11',
 'c12',
 'c13',
 'c14',
 'c15',
 'c16',
 'c17',
 'c18',
 'c19',
 'c20',
 'c21',
 'c22',
 'c23',
 'c24',
 'c25',
 'c26',
 'c27',
 'c28',
 'c29',
 'c30',
 'c31',
 'c32',
 'c33',
 'c34',
 'c35',
 'c36',
 'c37',
 'c38',
 'c39']

In [15]:
s.show(vertical=True)

-RECORD 0-------------------
 c0  | 1.0                  
 c1  | -0.06104319928899... 
 c2  | 0.009567439505994422 
 c3  | -0.04067442450909083 
 c4  | 0.08572355279011215  
 c5  | -0.26554282181769306 
 c6  | -0.22548584412649042 
 c7  | -0.21048066958486944 
 c8  | -0.22799719207658325 
 c9  | -0.45478881667449267 
 c10 | 0.5628429015639078   
 c11 | -0.3276097260071061  
 c12 | 0.15213296349014394  
 c13 | -0.01001306443818... 
 c14 | 68fd1e64             
 c15 | 38a947a1             
 c16 | 0d595c43             
 c17 | 3952a0a2             
 c18 | 43b19349             
 c19 | fe6b92e5             
 c20 | b67de336             
 c21 | 0b153874             
 c22 | a73ee510             
 c23 | 3b08e48b             
 c24 | 873e6871             
 c25 | be326377             
 c26 | 4cc724bd             
 c27 | b28479f6             
 c28 | a897defd             
 c29 | fadb1467             
 c30 | 27c07bd6             
 c31 | fe182c3d             
 c32 | null                 
 c33 | null   

In [16]:
# convert the categorical variables to indices
indexer = StringIndexer(inputCol='c22', outputCol='c22_idx', handleInvalid='keep')
s = indexer.fit(s).transform(s)

In [17]:
indexer = StringIndexer(inputCol='c33', outputCol='c33_idx', handleInvalid='keep')
s = indexer.fit(s).transform(s)
s.select('c22','c22_idx').show()

+--------+-------+
|     c22|c22_idx|
+--------+-------+
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
|a73ee510|    0.0|
+--------+-------+
only showing top 20 rows



In [18]:
# make sure it kept the null values
s.count()

45877

In [28]:
# encode one-hot vectors
encoder = OneHotEncoderEstimator(inputCols=['c22_idx','c33_idx'], outputCols=['c22_OHE', 'c33_OHE'])
s = encoder.fit(s).transform(s)

In [29]:
s.show()

+---+--------------------+--------------------+--------+--------+-------+-------+-------------+-------------+
| c0|                  c2|                  c3|     c22|     c33|c22_idx|c33_idx|      c22_OHE|      c33_OHE|
+---+--------------------+--------------------+--------+--------+-------+-------+-------------+-------------+
|0.0| -0.2734937899654927|-0.04722750963646816|a73ee510|    null|    0.0|    3.0|(3,[0],[1.0])|    (3,[],[])|
|0.0| -0.2734937899654927|-0.03838908232395208|a73ee510|    null|    0.0|    3.0|(3,[0],[1.0])|    (3,[],[])|
|0.0| -0.2655903267941605| 0.08358121458876974|a73ee510|b1252a9d|    0.0|    0.0|(3,[0],[1.0])|(3,[0],[1.0])|
|0.0| -0.2629558390703831| -0.0525305660239778|a73ee510|    null|    0.0|    3.0|(3,[0],[1.0])|    (3,[],[])|
|0.0| -0.2708593022417153|-0.04369213871146173|a73ee510|a458ea53|    0.0|    2.0|(3,[0],[1.0])|(3,[2],[1.0])|
|1.0| -0.2708593022417153| 0.04999519080120866|a73ee510|a458ea53|    0.0|    2.0|(3,[0],[1.0])|(3,[2],[1.0])|
|0.0| -0.2

In [33]:
# grab a column to see what it is like
c = s.select('c22_OHE').collect()[0][0]

In [34]:
type(c)

pyspark.ml.linalg.SparseVector

In [35]:
c.numNonzeros()

1

In [37]:
c.size

3

In [38]:
c.toArray()

array([1., 0., 0.])

In [40]:
# can convert to numpy array easily
type(c.toArray())

numpy.ndarray

In [43]:
s = s.drop('c22_idx', 'c33_idx')

In [44]:
s.show()

+---+--------------------+--------------------+--------+--------+-------------+-------------+
| c0|                  c2|                  c3|     c22|     c33|      c22_OHE|      c33_OHE|
+---+--------------------+--------------------+--------+--------+-------------+-------------+
|0.0| -0.2734937899654927|-0.04722750963646816|a73ee510|    null|(3,[0],[1.0])|    (3,[],[])|
|0.0| -0.2734937899654927|-0.03838908232395208|a73ee510|    null|(3,[0],[1.0])|    (3,[],[])|
|0.0| -0.2655903267941605| 0.08358121458876974|a73ee510|b1252a9d|(3,[0],[1.0])|(3,[0],[1.0])|
|0.0| -0.2629558390703831| -0.0525305660239778|a73ee510|    null|(3,[0],[1.0])|    (3,[],[])|
|0.0| -0.2708593022417153|-0.04369213871146173|a73ee510|a458ea53|(3,[0],[1.0])|(3,[2],[1.0])|
|1.0| -0.2708593022417153| 0.04999519080120866|a73ee510|a458ea53|(3,[0],[1.0])|(3,[2],[1.0])|
|0.0| -0.2629558390703831|0.039389078026189364|a73ee510|    null|(3,[0],[1.0])|    (3,[],[])|
|1.0|0.018934347373797816|                null|a73ee510|5840

In [45]:
s.write.format('parquet').save('data/sample.parquet')

In [9]:
for i,c in enumerate(train.columns[1:14]):
    train = train.withColumn(c, (train[c] - means[i]) / stdevs[i])

In [10]:
stats3 = train[train.columns[1:14]].describe()
stats3.show(vertical=True)

-RECORD 0-----------------------
 summary | count                
 c1      | 5851479              
 c2      | 10517430             
 c3      | 8259890              
 c4      | 8205798              
 c5      | 10229986             
 c6      | 8113327              
 c7      | 10051997             
 c8      | 10511722             
 c9      | 10051997             
 c10     | 5851479              
 c11     | 10051997             
 c12     | 2421098              
 c13     | 8205798              
-RECORD 1-----------------------
 summary | mean                 
 c1      | -7.97593112269365... 
 c2      | 5.828207091676494... 
 c3      | 3.909513938232698... 
 c4      | -3.74083514832785... 
 c5      | 9.537483029736827... 
 c6      | 3.244443654866041... 
 c7      | 9.026366645182133... 
 c8      | -4.24857390976072... 
 c9      | 6.576557065533054... 
 c10     | 1.379353179119628... 
 c11     | 4.190010407219867... 
 c12     | -1.01941175954099... 
 c13     | -4.75671009901150... 
-RECORD 2-

# Experimenting with RDDs

In [16]:
toyTrainRDD = sc.textFile('data/toy_train.txt')

In [17]:
toyTrainRDD.collect()[0]

'0\t\t1\t12\t1\t62526\t\t\t3\t\t\t\t\t3\t05db9164\t08d6d899\t9143c832\tf56b7dd5\t25c83c98\t7e0ccccf\t89a13b6b\t0b153874\t7cc72ec2\tc5fe5cb9\tfa365cf9\tae1bb660\td2420e4c\tb28479f6\tbffbd637\tbad5ee18\t07c540c4\tbbf70d82\t\t\t0429f84b\t\t32c7478e\tc0d61a5c\t\t'

In [21]:
line = toyTrainRDD.collect()[0]
line

'0\t\t1\t12\t1\t62526\t\t\t3\t\t\t\t\t3\t05db9164\t08d6d899\t9143c832\tf56b7dd5\t25c83c98\t7e0ccccf\t89a13b6b\t0b153874\t7cc72ec2\tc5fe5cb9\tfa365cf9\tae1bb660\td2420e4c\tb28479f6\tbffbd637\tbad5ee18\t07c540c4\tbbf70d82\t\t\t0429f84b\t\t32c7478e\tc0d61a5c\t\t'

In [22]:
line = line.split('\t')
line

['0',
 '',
 '1',
 '12',
 '1',
 '62526',
 '',
 '',
 '3',
 '',
 '',
 '',
 '',
 '3',
 '05db9164',
 '08d6d899',
 '9143c832',
 'f56b7dd5',
 '25c83c98',
 '7e0ccccf',
 '89a13b6b',
 '0b153874',
 '7cc72ec2',
 'c5fe5cb9',
 'fa365cf9',
 'ae1bb660',
 'd2420e4c',
 'b28479f6',
 'bffbd637',
 'bad5ee18',
 '07c540c4',
 'bbf70d82',
 '',
 '',
 '0429f84b',
 '',
 '32c7478e',
 'c0d61a5c',
 '',
 '']

In [12]:
def parse(line):
    line = line.split('\t')
    key = line[0]
    values = line[1:]
    for i,v in enumerate(values[:13]):
        v = 
    
    return key, values

In [14]:
toyTrainRDD = toyTrainRDD.map(parse)

toyTrainRDD.collect()[0]

('0',
 ('',
  '1',
  '12',
  '1',
  '62526',
  '',
  '',
  '3',
  '',
  '',
  '',
  '',
  '3',
  '05db9164',
  '08d6d899',
  '9143c832',
  'f56b7dd5',
  '25c83c98',
  '7e0ccccf',
  '89a13b6b',
  '0b153874',
  '7cc72ec2',
  'c5fe5cb9',
  'fa365cf9',
  'ae1bb660',
  'd2420e4c',
  'b28479f6',
  'bffbd637',
  'bad5ee18',
  '07c540c4',
  'bbf70d82',
  '',
  '',
  '0429f84b',
  '',
  '32c7478e',
  'c0d61a5c',
  '',
  ''))